https://www.cnblogs.com/edkong/p/16256620.html

collate_fn笼统的说就是用于整理数据，通常我们不需要使用.

其应用的情形是：各个数据长度不一样的情况，比如第一张图片大小是28 * 28,第二张是50 * 50，这样的话就如果不自己写collate_fn，而使用默认的，就会报错。

In [36]:
import torch
import torch.utils.data as Data
import numpy as np

# 便于结果得以复现，如何设置 seed
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [38]:
class mydataset(Data.Dataset):
    def __init__(self,train_inputs,train_targets):#必须有
        super(mydataset,self).__init__()
        self.inputs=train_inputs
        self.targets=train_targets
        
    def __getitem__(self, index):#必须重写
        return self.inputs[index],self.targets[index]
        
    def __len__(self):#必须重写
        return len(self.targets)

#构造训练数据
datax=torch.randn(4,3)#构造4个输入
datay=torch.empty(4).random_(2)#构造4个标签

#制作dataset
dataset=mydataset(datax,datay)

dataloader=Data.DataLoader(dataset,batch_size=2)

print(datax)
print(datay)
print(dataset)

batch=[dataset[0],dataset[1]]  # 所以才说和你dataset中get_item的定义有关。
print(batch)

tensor([[-0.9724,  0.0771,  1.3938],
        [-0.3671, -1.2116,  0.0274],
        [-0.7537, -1.4364, -0.2168],
        [-0.6196, -0.2774, -0.7631]])
tensor([0., 1., 1., 1.])
[(tensor([-0.9724,  0.0771,  1.3938]), tensor(0.)), (tensor([-0.3671, -1.2116,  0.0274]), tensor(1.))]


### 初始化 tensor

torch.empty + random_

In [32]:
# torch.empty创建的张量是未初始化的，即其值是随机的，取决于内存中的内容。因此，如果需要具有特定值的张量，应该使用其他函数进行初始化，如torch.zeros、torch.ones或torch.rand。
# 该函数的作用是快速创建指定大小的张量，但不保证张量中的数值是有意义的，需要用户自行初始化或填充数据。
print(torch.empty(4))

# `.random_()` 方法会生成指定范围内的随机数，并将这些随机数填充到张量中，覆盖原来的数据。这样可以用来对张量进行随机初始化或随机重置。
# 语法如下：
# torch.Tensor.random_(from=0, to=None, *, generator=None)，- `from`：指定生成随机数的下界（包含）。- `to`：指定生成随机数的上界（不包含）。
print(torch.empty(4).random_(2))
print(torch.empty(4).random_(3))

# 创建一个形状为(2, 3)的未初始化张量
x = torch.empty(2, 3)
print(x)

# 对张量进行计算
y = x + 5
print(y)

tensor([-2.1644e+09,  1.0860e-42,  3.0761e-01,  0.0000e+00])
tensor([1., 0., 0., 1.])
tensor([2., 0., 1., 2.])
tensor([[5., 5., 5.],
        [5., 5., 5.]])
tensor([[10., 10., 10.],
        [10., 10., 10.]])


In [9]:
it=iter(dataloader)
nex=next(it)#我们展示第一个batch经过collate_fn之后的输出结果
print(nex)

[tensor([[-1.4122, -0.4085, -1.8341],
        [ 0.7787,  1.5985, -0.7262]]), tensor([0., 1.])]


In [10]:
# a simple custom collate function, just to show the idea
# `batch` is a list of tuple where first element is input tensor and the second element is corresponding label
def my_collate(batch):
    inputs=[data[0].tolist() for data in batch]
    target = torch.tensor([data[1] for data in batch])
    return [data, target]

dataloader=Data.DataLoader(dataset,batch_size=2,collate_fn=my_collate)

print(datax)
print(datay)

tensor([[-1.4122, -0.4085, -1.8341],
        [ 0.7787,  1.5985, -0.7262],
        [ 0.2007, -0.8186,  0.8232],
        [-0.3312, -1.4139, -2.2054]])
tensor([0., 1., 0., 1.])


In [11]:
it=iter(dataloader)
nex=next(it)
print(nex)

NameError: name 'data' is not defined